# Inferring parameters for the ToR-ORd model

In this notebook, we use PINTS to create noisy data for a ToR-ORd [$^{[1]}$](#Sources) system and then perform inference on the model
to try and retrieve the parameters. This should be compared with results from other notebooks, such as the FitzHugh-Nagumo and Beeler-Reuter
notebooks.

## Creating the ForwardModel

### Variable descriptions

We need to set up the ODE system first. This is a large system and so we will not go through all the equations and varibles,
but we will give a description of all the currents and fluxes in the model and the important parameters (to be inferred).

|variable|description|ions|
|---|-------|---|
|$V$|membrane potential|all|
|$I_{Na}$|fast sodium current|$Na^{+}$|
|$I_{NaL}$|late sodium current|$Na^{+}$|
|$I_{to}$|transient outward potassium current|$K^{+}$|
|$I_{CaL}$|L-type calcium current|$Ca^{2+}$|
|$I_{K_r}$|rapid delayed rectifier potassium current|$K^{+}$|
|$I_{K_s}$|slow delayed rectifier potassium current|$K^{+}$|
|$I_{K_1}$|inward rectifier current|$K^{+}$|
|$I_{NaCa}$|sodium/calcium exchanger (NCX)|$Na^{+}$, $Ca^{2+}$|
|$I_{NaK}$|sodium/potassium pump|$Na^{+}$, $K^{+}$|
|$I_{Kb}$|background potassium current|$K^{+}$|
|$I_{Nab}$|background sodium current|$Na^{+}$|
|$I_{Cab}$|background calcium current|$Ca^{2+}$|
|$I_{pCa}$|calcium pump|$Ca^{2+}$|
|$I_{ClCa}$|calcium-sensitive chloride current|$Cl^{-}$|
|$I_{Clb}$|background chloride current|$Cl^{-}$|
|$J_{rel}$|calcium flux through ryanodine receptor|$Ca^{2+}$|
|$J_{up}$|calcium uptake into network sarcoplasmic reticulum via SERCA|$Ca^{2+}$|
|$J_{leak}$|calcium leak out of network sarcoplasmic reticulum|$Ca^{2+}$|
|$J_{tr}$|calcium NSR to JSR calcium translocation|$Ca^{2+}$|
|$J_{diff,Na}$|sodium diffusion flux between myoplasm and subspace|$Na^{+}$|
|$J_{diff,K}$|potassium diffusion flux between myoplasm and subspace|$K^{+}$|
|$J_{diff}$|calcium diffusion flux between myoplasm and subspace|$Ca^{2+}$|
|CMDN|calmodulin buffer|$Ca^{2+}$|
|TRPN|troponin buffer|$Ca^{2+}$|
|CSQN|calsequestrin buffer|$Ca^{2+}$|
|BSR|anionic sarcoplasmic reticulum binding sites for calcium|$Ca^{2+}$|
|BSL|anionic sarcolemmal binding sites for calcium|$Ca^{2+}$|
|CaMKII|calmodulin-dependent protein kinase II|$Ca^{2+}$|

All of the above currents have corresponding conductances, and the fluxes have corresponding multipliers. The conductances we will
be most interested in for the purpose of inference are listed in the table below.

|conductance|ToR-ORd value [$^{[1]}$](#Sources)|ion|reasoning for inference|
|---|---|---|-------|
|$G_{Na}$|$11.7802$|$Na^{+}$|Multiple changes were made: the Grandi [$^{[2]}$](#Sources) model was used with ORd [$^{[3]}$](#Sources) CaMKII phosphorylation.|
|$P_{Cab}$|$8.3757 \times 10^{-5}$|$Ca^{2+}$|The L-type calcium current had the largest revision of them all compared to the ORd model.|
|$G_{K_r}$|$0.0321$|$K^{+}$|This was changed to have a Markovian [$^{[4]}$](#Sources) formulation, as originally parameter inference could not fit the action potential.|
|$G_{ClCa}$|$0.2843$|$Cl^{-}$|This current was added for the purpose of fitting to the shape of the action potential.|
|$G_{Clb}$|$0.00198$|$Cl^{-}$|This current was added for the purpose of fitting to the shape of the action potential.|

We will not go into any major detail with the equations, but the main membrane potential ODE is a sum of all the above
currents added to a stimulating current, $I_{stim}$ which provides the action potential shape. We also note here that the
ToR-ORd model has $43$ time dependent variables forming a system of $43$ ordinary differential equations.

|variable group|list of variables|ions|
|---|---|---|
|Membrane potential|$V$|all|
|Ion compartment concentrations|$[Na]_{i}, [Na]_{ss}, [K]_{i}, [K]_{ss}, [Ca]_{i}, [Ca]_{ss}, [Ca]_{nsr}, [Ca]_{jsr}$|$Na^{+}, K^{+}, Ca^{2+}$|
|$I_{Na}$ gating variables|$m, h_{p}, h, j, j_{p}$|$Na^{+}$|
|$I_{NaL}$ gating variables|$m_L, h_L, h_{Lp}$|$Na^{+}$|
|$I_{to}$ gating variables|$a, i_{F}, i_{S}, a_{p}, i_{Fp}, i_{Sp}$|$K^{+}$|
|$I_{CaL}$ gating variables|$d, f_{f}, f_{s}, f_{caf}, f_{cas}, j_{ca}, n_{ca}, n_{cai}, f_{fp}, f_{cafp}$|$K^{+}$|
|$I_{K_r}$ gating variables|$C_{0}, C_{1}, C_{2}, O, I$|$K^{+}$|
|$I_{K_s}$ gating variables|$x_{s1}, x_{s2}$|$K^{+}$|
|$J_{rel}$|$J_{relp}, J_{relnp}$|$Ca^{2+}$|
|CaMKII buffer|$CaMK_{t}$|$Ca^{2+}$|

### Initial conditions

We will use the following initial conditions, representing an endocardial cell.

In [1]:
initial_conditions = [-88.8691566357934,12.0996647655188,12.1000028563765,142.412524737626,142.412481425842,7.45541572746214e-05,
                      6.50418928341426e-05,1.53037019085812,1.52803094224238,0.000787657400526199,0.674096901201792,0.830658198588696,
                      0.830466744399495,0.830093612199637,0.000159670117055769,0.528261721740178,0.288775833197764,0.000944249645410894,
                      0.999616956857814,0.593680589620082,0.000481107253796778,0.999616964658062,0.654092074678260,8.86091322819384e-29,
                      0.999999992783113,0.938965241412012,0.999999992783179,0.999900458262832,0.999977476316330,0.000492094765239740,
                      0.000833711885764158,0.999999992566681,0.999999992766279,0.247156543918935,0.000175017075236424,3.90843796133124e-24,
                      0.0110752904836162,0.998073652444028,0.000844745297078649,0.000698171876592920,0.000370404872169913,1.30239063420973e-05,
                      -1.88428892080206e-22]

In [2]:
import pints
import numpy as np
import scipy.integrate as si
import matplotlib.pyplot as plt

### PINTS implementation of the model

In addition to implementing the system, we give the option in the class to specify fixed parameters, if we wish to 
reduce the dimensions of the parameter space. Additionally, as we realistically would only be able to measure the membrane potential,
$V$, and the calcium concentration, $c$, we will use these as the only outputs of the system.

The system of equations from the model have been adapted from the original MATLAB code from the ToR-ORd github repository [$^{[5]}$](#Sources).

In [ ]:
class ToRORdModel(pints.ForwardModel):

    def __init__(self, initial_conditions, stimulating_current, fixed_params=None, outputs=["V", "c"],
                 rtol=10**(-3), atol=10**(-6)):
        """Initialises the forward model with certain fixed parameters passed to the method
        as key-value pairs.
        """
        super(ToRORdModel, self).__init__()
        assert len(initial_conditions) == 43
        self._initial_conditions = initial_conditions

        assert len(stimulating_current) == 2
        self._stim_duration, self._stim_amplitude = tuple(stimulating_current)
        self._param_names = ["G_Na", "P_Cab", "G_Kr", "G_ClCa", "G_Clb"]
        self._variable_names = ["V", "Na_i", "Na_ss", "K_i", "K_ss", "Ca_i", "Ca_ss", "Ca_nsr", "Ca_jsr",
                                "m", "hp", "h", "j", "jp",
                                "mL", "hL", "hLp",
                                "a", "iF", "iS", "ap", "iFp", "iSp",
                                "d", "ff", "fs", "fcaf", "fcas", "jca", "nca", "nca_i", "ffp", "fcafp",
                                "xs1", "xs2",
                                "Jrel_np", "CaMKt",
                                "ikr_c0", "ikr_c1", "ikr_c2", "ikr_o", "ikr_i",
                                "Jrel_p"]
        self._fixed_params = {}
        if fixed_params is not None:
            for k in fixed_params.keys():
                if k in self._param_names:
                    self._fixed_params[k] = fixed_params[k]

        assert 1 <= len(outputs) <= 43
        assert set(outputs).issubset(set(self._variable_names))
        self._outputs = outputs
        self._rtol = rtol
        self._atol = atol
        # If we wish to specific current or flux multipliers, we do so in the dict below (e.g. for blocking channels)
        self.multipliers = {"I_CaL": 1, "I_Na": 1, "I_to": 1, "I_NaL": 1, "I_Kr": 1, "I_Ks": 1, "I_K1": 1, "I_Kb": 1,
                             "I_NaCa": 1, "I_NaK": 1, "I_Nab": 1, "I_Cab": 1, "I_pCa": 1, "I_CaCl": 1, "I_Clb": 1,
                             "J_rel": 1, "J_up": 1}
        # If we wish to change the extracellular concentrations or subspace fractions, we do so here
        self.concs_and_fractions = {"Na_o": 140.0, "Ca_o": 1.8, "K_o": 5.0, "ICaL_fractionSS": 0.8, "INaCa_fractionSS": 0.35}

    def get_param_names(self):
        """Returns the parameter names.
        """
        return self._param_names

    def get_unfixed_param_names(self):
        """Returns the unfixed parameters
        """
        output_list = self._param_names.copy()
        for fixed_param in self._fixed_params.keys():
            output_list.remove(fixed_param)
        return output_list

    def n_outputs(self):
        """Returns number of model outputs, default to just V and c
        """
        return len(self._outputs)

    def n_parameters(self):
        """Returns number of parameters to be inferred, i.e. all non-fixed parameters.
        """
        return len(self._param_names) - len(self._fixed_params)

    def n_fixed_parameters(self):
        """Returns number of parameters we fix.
        """
        return len(self._fixed_params)

    def simulate(self, parameters, times):
        """This inherits from pints.ForwardModel, performs one forward simulation of the model.
        The list `parameters` contains only the parameters we are inferring.
        """
        G_Na, P_Cab, G_Kr, G_ClCa, G_Clb = self._get_all_params(parameters)
        t_span = [times[0], times[len(times) - 1]]
        y0 = self._initial_conditions
        sol = si.solve_ivp(fun=self._rhs, t_span=t_span, y0=y0, 
                           t_eval=times, args=(G_Na, P_Cab, G_Kr, G_ClCa, G_Clb,
                                               self._stim_duration, self._stim_amplitude, 
                                               self.multipliers, self.concs_and_fractions),
                           rtol=self._rtol, atol=self._atol)

        # Now we find which outputs we wish to use
        output_indices = [self._variable_names.index(variable) for variable in self._outputs]
        return np.array([sol.y[index, :] for index in output_indices]).transpose()

    def _get_all_params(self, parameters):
        """We need a list of 5 parameters, so we check to see which parameters are fixed, and which are not.
        We increment j each time a non-fixed parameter is reached.
        """
        all_parameters = []
        j = 0
        for param in self._param_names:
            if param in self._fixed_params.keys():
                all_parameters.append(self._fixed_params[param])
            else:
                all_parameters.append(parameters[j])
                j += 1
        return tuple(all_parameters)

    @staticmethod
    def _rhs(t, y, G_Na, P_Cab, G_Kr, G_ClCa, G_Clb, stim_duration, stim_amplitude, multipliers, concs_and_fractions):
        """The right hand side function of the ToR-ORd system. This is in a separate file as it is 
        a very long system of equations.
        """
        torord = ModelTorord(t, y, G_Na, P_Cab, G_Kr, G_ClCa, G_Clb, stim_duration, stim_amplitude, multipliers, concs_and_fractions)
        return torord.simulate()
        # Variables
        V, m, h, j, d, f, x, c = tuple(list(y))

        # Currents
        I_Na = (g_Na * (m**3) * h * j + g_NaC) * (V - V_Na)
        I_K = g_K1 * (exp(0.04 * (V - V_K1)) - 1) / (exp(0.08 * (V - V_K2)) + exp(0.04 * (V - V_K2))) + g_K2 * (V - V_K3) / (1 - exp(-0.04 * (V - V_K3)))
        I_x = g_x * x * (exp(0.04 * (V - V_x1)) - 1) / exp(0.04 * (V - V_x2))
        I_Ca = g_Ca * d * f * (V - V_Ca + 13.0287 * (log(c) - 7 * log(10)))

        # Stimulation parameters
        if (t - stim_start) % stim_period <= stim_duration:
            I_stim = stim_amplitude
        else:
            I_stim = 0

        # Membrane potential ODE
        V_dot = - (I_Na + I_K + I_x + I_Ca) + I_stim

        # Gating variable ODEs
        m_dot = (V + 47) / (1 - exp(-0.1 * (V + 47))) * (1 - m) - 40 * exp(-0.056 * (V + 72)) * m
        h_dot = 0.126 * exp(-0.25 * (V + 77)) * (1 - h) - 1.7 / (1 + exp(-0.082 * (V + 22.5))) * h
        j_dot = 0.055 * exp(-0.25 * (V + 78)) / (1 + exp(-0.2 * (V + 78))) * (1 - j) - 0.3 / (1 + exp(-0.1 * (V + 32))) * j
        d_dot = 0.095 * exp(-0.01 * (V + -5)) / (exp(-0.072 * (V + -5)) + 1) * (1 - d) - 0.07 * exp(-0.017 * (V + 44)) / (exp(0.05 * (V + 44)) + 1) * d
        f_dot = 0.012 * exp(-0.008 * (V + 28)) / (exp(0.15 * (V + 28)) + 1) * (1 - f) - 0.0065 * exp(-0.02 * (V + 30)) / (exp(-0.2 * (V + 30)) + 1) * f
        x_dot = 0.0005 * exp(0.083 * (V + 50)) / (exp(0.057 * (V + 50)) + 1) * (1 - x) - 0.0013 * exp(-0.06 * (V + 20)) / (exp(-0.04 * (V + 20)) + 1) * x

        # Calcium concentration ODE
        c_dot = 0.07 * (1 - c) - I_Ca
        return np.array([V_dot, m_dot, h_dot, j_dot, d_dot, f_dot, x_dot, c_dot])

## Sources

$^{[1]}$ J. Tomek *et al.*, Development, calibration, and validation of a novel human ventricular myocyte model in health, disease, and drug block, eLife **8** (2019), e48890, https://doi.org/10.7554/eLife.48890.

$^{[2]}$ E. Grandi *et al.*, A Novel Computational Model of the Human Ventricular Action Potential and Ca Transient, J. Mol. Cell Cardiol. **48** (2010), pp. 112, https://pubmed.ncbi.nlm.nih.gov/19835882/.

$^{[3]}$ T. O’Hara *et al.*, Simulation of the Undiseased Human Cardiac Ventricular Action Potential: Model Formulation and Experimental 
Validation, PLoS Comput. Biol.** **7 (2011), e100206, https://doi.org/10.1371/journal.pcbi.10020611.

$^{[4]}$ Y. Lu *et al.*, Effects of premature stimulation on HERG K+ channels **537** (2001), pp. 843-851, https://pubmed.ncbi.nlm.nih.gov/11744759/.

$^{[5]}$ J. Tomek, Repository for the cardiac model ToR-ORd, Github, 4ffab13, https://github.com/jtmff/torord. Date first accessed: 2024-04-22. 